#### ???
* Что такое синтаксическое дерево
* Почему регулярные выражения так называются
* Чем компилятор отличается от интерперетора

In [ ]:
from datetime import datetime
start_time = datetime.now()
def time_elapsed():
    global start_time
    print(datetime.now() - start_time)

#### Что мы будем использовать?

Python и его библиотеки
* https://docs.python.org/3/library/dis.html
* https://docs.python.org/3/library/ast.html
* https://greentreesnakes.readthedocs.io/en/latest


* [lark-parser](https://github.com/lark-parser/lark) - библиотека для построения парсера по спецецификации
* [astpretty](https://github.com/asottile/astpretty) - вывод ast на экран


* Утилиты:
* [funcy](https://github.com/Suor/funcy)
* [more-itertools](https://github.com/more-itertools/more-itertools)

*Отступление*
* [facesofopensource.com](https://www.facesofopensource.com/guido-van-rossum-2/)


#### Нужно ли это разработчикам?
???

### Определения

[Язык программирования](https://ru.wikipedia.org/wiki/%D0%AF%D0%B7%D1%8B%D0%BA_%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F) -  формальный язык, предназначенный для записи компьютерных программ с набором набор лексических, синтаксических и семантических правил.

[Формальный язык](https://ru.wikipedia.org/wiki/%D0%A4%D0%BE%D1%80%D0%BC%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9_%D1%8F%D0%B7%D1%8B%D0%BA) - множество слов (цепочек символов) над конечным алфавитом, определенных посредсвом некторых правил



### [Иерархия классов](https://en.wikipedia.org/wiki/Chomsky_hierarchy)
![Chomsky-hierarchy](assets/640px-Chomsky-hierarchy.svg.png)

### Процесс компиляции

![flow](assets/parser_flow.diag.svg)



### Токенизация

In [ ]:
from io import BytesIO
import tokenize

In [ ]:
tokenize.tokenize?

In [ ]:
data = BytesIO(b"""
if x > y:
  r = x - y * 2
else:
  r = x + y
""").readline

for tok in tokenize.tokenize(data):
    print(tok)

#### Что такое AST?

`1 + 2 * 3`

```
     +
    / \
   1   *
      / \
     2   3
```

```
EXPR -> NUM | EXPR + EXPR | EXPR * EXPR
NUM -> 0|1|2|...|9
```


#### AST vs Parse Tree
*опционально*

In [ ]:
import ast
import astpretty

* https://docs.python.org/3/library/ast.html
* https://greentreesnakes.readthedocs.io/en/latest

In [ ]:
ast.parse?

In [ ]:
node = ast.parse("""
if x > y:
  r = x - y * 2
else:
  r = x + y
""")
print(node)

In [ ]:
# TODO: work with `node`

In [ ]:
astpretty.pprint(node)

In [ ]:
time_elapsed()

#### Пробуем написать свой парсер

In [ ]:
import more_itertools
from pprint import pprint

In [ ]:
print(tokens := '1 + 2 + 3 + 4'.split())
stream = more_itertools.peekable(tokens)

In [ ]:
print('peek', stream.peek())
print('next', next(stream))
print('peek', stream.peek())
print('peek', stream.peek())
print('next', next(stream))
print('next', next(stream))

In [ ]:
def mynode(typ, val):
    assert val is not None
    return {
        'typ': typ,
        'val': val,
    }

####  Определим грамматику
```
EXPR -> ...
NUM -> ...
```

In [ ]:
def consume_int(stream):
    if stream.peek('').isdigit():
        return int(next(stream))
    return None

def consume_tok(tokens, stream):
    tok = stream.peek(None)
    if tok is not None and tok in tokens:
        return next(stream)
    return None

def parse_expr(stream):
    lhs = mynode('num', consume_int(stream))

    op = consume_tok(['+', '-'], stream)
    if op is None:
        return lhs
    rhs = parse_expr(stream)
    assert rhs is not None
    return mynode('op', [lhs, op, rhs])

In [ ]:
node = parse_expr(more_itertools.peekable('1 + 2 + 3 + 4'.split()))
pprint(node)

In [ ]:
def evaluate(node):
    ops = {
        '+': lambda x, y: x + y,
        '-': lambda x, y: x - y,
    }
    if node['typ'] == 'op':
        lchild, op, rchild =  node['val']
        return ops[op](evaluate(lchild), evaluate(rchild))
    if node['typ'] == 'num':
        return node['val']
    assert 'unknown type', node


In [ ]:
evaluate(node)

In [ ]:
node = parse_expr(more_itertools.peekable('1 + 2 - 3 + 4'.split()))
pprint(node)

#### Что выведет `evaluate` ?
```
  +
 / \
1   -
   / \
  2   +
     / \
    3   4
```

In [ ]:
evaluate(node)

#### Грамматика версия 2
```
EXPR -> NUM | EXPR OP NUM
NUM -> 0..9
```

In [ ]:
def parse_expr2(stream):
    lhs = parse_expr2(stream)
    
    op = consume_tok(['+', '-'], stream)
    if op is None:
        return lhs
    rhs = mynode('num', consume_int(stream))
    assert rhs is not None
    return mynode('op', [lhs, op, rhs])

#### DO NOT RUN !!!

In [ ]:
node = parse_expr2(more_itertools.peekable('1 + 2 - 3 + 4'.split()))
pprint(node)

#### Грамматика версия 3 
```
EXPR -> NUM {[+-] NUM}
NUM -> 0..9
```

In [ ]:
def parse_expr3(stream):
    lhs = mynode('num', consume_int(stream))
        
    assert lhs is not None
    res = lhs
    
    while True:
        op = consume_tok(['+', '-'], stream)
        if op is None:
            break

        rhs = mynode('num', consume_int(stream))
        assert rhs is not None
        
        res = mynode('op', [res, op, rhs])
    return res


In [ ]:
node = parse_expr3(more_itertools.peekable('1 + 2 - 3 + 4'.split()))
pprint(node)

#### Результат
```
        + 
       / \
      -   4
     / \
    +   3
   / \ 
  1   2 
 
```

In [ ]:
evaluate(node)

In [ ]:
time_elapsed()

### Что делать с AST?

* https://docs.python.org/3/library/dis.html
* [Understanding Python Bytecode](https://towardsdatascience.com/understanding-python-bytecode-e7edaae8734d)
* [How to patch Python bytecode](https://rushter.com/blog/python-bytecode-patch/)

In [ ]:
import dis

In [ ]:
compile?

In [ ]:
node = ast.parse("""
x = a + b
if x > 0:
    print(x)
""")

In [ ]:
code = compile(node, '<foo>', "exec")

In [ ]:
code

In [ ]:
def pcodeobj(code):
    for attr in dir(code):
        if attr.startswith('co_'):
            print("\t%s = %s" % (attr, code.__getattribute__(attr)))


In [ ]:
pcodeobj(code)

In [ ]:
dis.dis(code)

### Почему байткод быстрее AST-интепретации?

* простой, меньше накладных расходов в интепретаторе
* легче оптимизировать
* дружественный к кешу

### Ruby 1.8 vs 1.9

http://rubychan.de/share/yarv_speedups.html


*Отступление*

Principle of locality


### Процесс компиляции (еще раз)

![flow](assets/parser_flow.diag.svg)



### Rust Example
https://blog.rust-lang.org/2016/04/19/MIR.html

### Let's Code ...

In [ ]:
import typing
import dis
import ast

from types import CodeType

import struct

from io import BytesIO
from dataclasses import dataclass
from collections import defaultdict

from lark import Lark, Transformer, v_args
from funcy import collecting

https://github.com/lark-parser/lark

#### Что напишем?
* https://en.wikipedia.org/wiki/Euclidean_algorithm
* https://en.wikipedia.org/wiki/Fibonacci_number

* Наш язык поддержит целые числа и операции `+ - * /`
* Операторы сравнения `<, >, >=, ...`
* Условные и безусловные переходы (aka `if goto`)
* Ввод-вывод

In [ ]:
def gdc(a, b):
    while a != b:
        if a > b:
            a -= b
        else:
            b -= a
    return a

assert gdc(12, 8) == 4
assert gdc(56, 42) == 14
assert gdc(1, 10) == 1
del gdc

In [ ]:
def fib(n):
    a = 1
    b = 1
    for _ in range(n):
        c = a + b
        a = b
        b = c
    return a 

assert fib(1) == 1
assert fib(2) == 2
assert fib(3) == 3
assert fib(4) == 5
assert fib(5) == 8
assert fib(6) == 13
del fib

In [ ]:
text = """
"""



In [ ]:
grammar = """
    start: _NL* statement (_NL+ statement )* _NL*
    
    ?statement: ...

    ?expr: sum
           | sum CMP sum -> binop  
    
    ?sum: product
        | sum ADDOP product -> binop

    CMP: "="|">="|"<="|"/="|">"|"<"
    ADDOP: "+"|"-"
    MULOP: "*"|"/"
    
    ?product: atom
        | product MULOP atom -> binop
        
    ?atom: NUMBER           -> number
         | NAME             -> var
         | "(" sum ")"

    %import common.CNAME -> NAME
    %import common.NEWLINE -> _NL
    %import common.NUMBER
    %import common.WS_INLINE
    %ignore WS_INLINE
"""

In [ ]:
parser = Lark(grammar, parser='lalr')

In [ ]:
parser.parse(text)

In [ ]:
print(parser.parse(text).pretty())

#### Опрделяем таргет

In [ ]:
@dataclass
class PythonInst:
    mnemonic: str
    argument: typing.Any = 0
    argtype: str = None
    
    def size(self):
        return ...

    def to_binary(self, ctx, writer):
        operand = 0
        if self.argtype is None:
            operand = self.argument
        else:
            operand = ctx[self.argtype][self.argument]
        opcode = dis.opmap[self.mnemonic]
        writer.write(struct.pack('BB', opcode, operand))
        
@dataclass
class LabelInst:
    label: str

    def size(self):
        return ...

    def to_binary(self, ctx, writer):
        ...

#### stdlib

In [ ]:
def read_ints():
    return tuple(int(v) for v in input('>> ').split())

def read_int():
    return int(input('> '))

In [ ]:
dis.dis(compile(ast.parse("""
x + y
"""), '<foo>', 'exec'))

https://docs.python.org/3/library/dis.html
* `CALL_FUNCTION`
* `JUMP_ABSOLUTE`
* `LOAD_CONST`
* `LOAD_NAME`
* `POP_JUMP_IF_TRUE`
* `POP_TOP`
* `RETURN_VALUE`
* `STORE_NAME`
* `UNPACK_SEQUENCE`
* `BINARY_MULTIPLY`
* `BINARY_TRUE_DIVIDE`
* `BINARY_ADD`
* `BINARY_SUBTRACT`
* `COMPARE_OP`

#### Пишем визитор

In [ ]:
op_mapping = {
    '*': PythonInst('BINARY_MULTIPLY', 0),
    '<': PythonInst('COMPARE_OP', dis.cmp_op.index('<')), 
    ...: ...,
}


@v_args(inline=True)
class VisitTree(Transformer):

    @collecting
    def number(self, num):
        yield PythonInst('LOAD_CONST', int(num), 'const')
        
    @collecting
    def var(self, name):
        ...
    
    @collecting
    def binop(self, lhs, op, rhs):
        ...
        
    @collecting
    def ask(self, *names):
        ...

    @collecting
    def answer(self, *names):
        ...

    @collecting
    def label(self, name):
        ...
    
    @collecting
    def goto(self, label, expr=None):
        ...
     
    @collecting       
    def assign(self, name, expr):
        ...

    @collecting
    def start(self, *statements):
        ...
    
parser = Lark(grammar, parser='lalr', transformer=VisitTree())

In [ ]:
parser.parse(text)

In [ ]:
def create_codeobj(instructions):
    
    constants = tuple(set(
        inst.argument
        for inst in instructions
        if isinstance(inst, PythonInst) and inst.argtype == 'const'
    ))
    
    names = tuple(set(
        inst.argument
        for inst in instructions
        if isinstance(inst, PythonInst) and inst.argtype == 'name'
    ))
    context = {
        'const': {
            k: i
            for i, k in enumerate(constants)
        },
        'name': {
            k: i
            for i, k in enumerate(names)
        },
        'label': {},
    }
    
    off = 0
    for inst in instructions:
        if isinstance(inst, LabelInst):
            context['label'][inst.label]  = off
        off += inst.size()
    
    
    for inst in instructions:
        if isinstance(inst, PythonInst):
            if inst.argtype == 'label':
                assert inst.argument in context['label']
    
    w = BytesIO()

    for inst in instructions:
        inst.to_binary(context, w)
    
    return CodeType(
        0, # argcount
        0, # posonlyargcount
        0, # kwonlyargcount
        0, # nlocals
        64, # stacksize
        64, # flags
        w.getvalue(), # codestring
        constants, # constants
        names, # names
        (), # varnames
        '<ok>', # filename
        '<module>', # name
        1, # firstlineno
        b'',# lnotab
    )

In [ ]:
code = create_codeobj(parser.parse(text))

In [ ]:
dis.dis(code)

In [ ]:
exec(code)

### Книги


Compilers:
![CompilersBook2ed](assets/CompilersBook2ed.png)

* [dragonbook](https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D0%BC%D0%BF%D0%B8%D0%BB%D1%8F%D1%82%D0%BE%D1%80%D1%8B:_%D0%BF%D1%80%D0%B8%D0%BD%D1%86%D0%B8%D0%BF%D1%8B,_%D1%82%D0%B5%D1%85%D0%BD%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D0%B8_%D0%B8_%D0%B8%D0%BD%D1%81%D1%82%D1%80%D1%83%D0%BC%D0%B5%D0%BD%D1%82%D1%8B) (aka "Компиляторы: принципы, технологии и инструменты")

* [craftinginterpreters](https://craftinginterpreters.com/contents.html)

* [Engineering: A Compiler](https://www.amazon.com/Engineering-Compiler-Keith-Cooper/dp/012088478X)


Python internals and advanced:
* [Inside The Python Virtual Machine](https://leanpub.com/insidethepythonvirtualmachine)
* [Intermediate Python](https://leanpub.com/intermediatepython)